### 【問題1】クロスバリデーション
事前学習期間では検証データをはじめに分割しておき、それに対して指標値を計算することで検証を行っていました。（ホールドアウト法）しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション（交差検証） を行います。分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割のためにscikit-learnにはKFoldクラスが用意されています。

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_curve, auc, accuracy_score)
from sklearn.model_selection import KFold

#csvファイルの読込
df_train = pd.read_csv("application_train.csv")
df_test = pd.read_csv("application_test.csv")

#使用する特徴量
features = ["AMT_INCOME_TOTAL", "REGION_RATING_CLIENT", "DAYS_BIRTH", "DAYS_ID_PUBLISH"]

#trainデータの特徴量と目的変数の抽出
X = df_train.loc[:,features].values
y = df_train.loc[:,"TARGET"].values

#train, testデータへの分割
X_init_train, X_init_test, y_init_train, y_init_test = train_test_split(X, y, test_size=0.2)

#標準化
scaler = StandardScaler()
scaler.fit(X_init_train)
X_train_transformed = scaler.transform(X_init_train)
X_test_transformed = scaler.transform(X_init_test)

#クロスバリデーション
accuracy_list = []
auc_list = []
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X_train_transformed):
    X_train_cross, X_test_cross = X_train_transformed[train_index], X_train_transformed[test_index]
    y_train_cross, y_test_cross = y_init_train[train_index], y_init_train[test_index]

    #ランダムフォレストにて学習
    rf = RandomForestClassifier()
    rf.fit(X_train_cross, y_train_cross)
    
    #精度予測
    y_pred = rf.predict(X_test_transformed)
    score = accuracy_score(y_init_test, y_pred)
    accuracy_list.append(score)

    #ROC予測
    y_pred = rf.predict_proba(X_test_transformed)
    fpr, tpr, thresholds = roc_curve(y_init_test, y_pred[:,1], pos_label=1)
    auc_list.append(auc(fpr, tpr))
    
#予測精度, AUCの表示
for i, acc in enumerate(accuracy_list):
    print("No.{}, ACC : {}".format(i, acc))
print("ACCの平均: {}".format(np.mean(accuracy_list)))
for i, auc in enumerate(auc_list):
    print("No.{}, AUC : {}".format(i, auc))
print("AUCの平均: {}".format(np.mean(auc_list)))

#test.csvの特徴量の抽出
X_test_features = df_test.loc[:,features].values

#test.csv特徴量の標準化
scaler = StandardScaler()
scaler.fit(X_test_features)
X_test_features_transformed = scaler.transform(X_test_features)

#test.csv特徴量からデフォルト予測
y_test_pred = rf.predict_proba(X_test_features_transformed)

#以下、Kaggle提出用データの作成
df_X_test_ID = df_test.loc[:,["SK_ID_CURR"]]
df_y_pred = pd.DataFrame(y_test_pred[:,1])
df_output = pd.concat([df_X_test_ID, df_y_pred], axis=1)
df_output.columns = ["SK_ID_CURR", "TARGET"]
df_output.to_csv('home_creditdata1.csv',index = False)

No.0, ACC : 0.9160528754694893
No.1, ACC : 0.9161829504251825
No.2, ACC : 0.9166056940311855
No.3, ACC : 0.9157276880802563
No.4, ACC : 0.9159390598832577
ACCの平均: 0.9161016535778742
No.0, AUC : 0.5507189623767521
No.1, AUC : 0.550369544217318
No.2, AUC : 0.5454073552687767
No.3, AUC : 0.5521501503113658
No.4, AUC : 0.5476827519428941
AUCの平均: 0.5492657528234213


## 【問題2】グリッドサーチ
これまで分類器のパラメータには触れず、デフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになります。機械学習の前提として、パラメータは状況に応じて最適なものを選ぶ必要があります。最適なパラメータを探していくことを パラメータチューニング と呼びます。パラメータチューニングをある程度自動化する単純な方法としては グリッドサーチ があります。

scikit-learnのGridSearchCVを使い、グリッドサーチを行うコードを作成してください。そして、ベースラインモデルに対して何らかしらのパラメータチューニングを行なってください。どのパラメータをチューニングするかは、使用した手法の公式ドキュメントを参考にしてください。

In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#csvファイルの読込
df_train = pd.read_csv("application_train.csv")
df_test = pd.read_csv("application_test.csv")

#使用する特徴量
features = ["AMT_INCOME_TOTAL", "REGION_RATING_CLIENT", "DAYS_BIRTH", "DAYS_ID_PUBLISH"]

#trainデータの特徴量と目的変数の抽出
X = df_train.loc[:,features].values
y = df_train.loc[:,"TARGET"].values

#train, testデータへの分割
X_init_train, X_init_test, y_init_train, y_init_test = train_test_split(X, y, test_size=0.2)

#標準化
scaler = StandardScaler()
scaler.fit(X_init_train)
X_train_transformed = scaler.transform(X_init_train)
X_test_transformed = scaler.transform(X_init_test)

#グリッドサーチ実行
param = {'max_depth':[5,None],
             'n_estimators':[120, 300]}

rf = GridSearchCV(RandomForestClassifier(),   param)
rf.fit(X_train_transformed, y_init_train) 

best_rf = rf.best_estimator_
print('.best_estimator_の内容:', best_rf)

scores = rf.cv_results_['mean_test_score']
params = rf.cv_results_['params']
for i in range(len(scores)):
    print(scores[i], params[i])
    
"""
.best_estimator_の内容: RandomForestClassifier(max_depth=5, n_estimators=120)
0.9190229586279912 {'max_depth': 5, 'n_estimators': 120}
0.9190229586279912 {'max_depth': 5, 'n_estimators': 300}
0.9156612791509164 {'max_depth': None, 'n_estimators': 120}
0.9157059945154163 {'max_depth': None, 'n_estimators': 300}
"""

.best_estimator_の内容: RandomForestClassifier(max_depth=5, n_estimators=120)
0.9194579038495918 {'max_depth': 5, 'n_estimators': 120}
0.9194579038495918 {'max_depth': 5, 'n_estimators': 300}
0.915628758577887 {'max_depth': None, 'n_estimators': 120}
0.9158563937489961 {'max_depth': None, 'n_estimators': 300}


"\n.best_estimator_の内容: RandomForestClassifier(max_depth=5, n_estimators=120)\n0.9190229586279912 {'max_depth': 5, 'n_estimators': 120}\n0.9190229586279912 {'max_depth': 5, 'n_estimators': 300}\n0.9156612791509164 {'max_depth': None, 'n_estimators': 120}\n0.9157059945154163 {'max_depth': None, 'n_estimators': 300}\n"

### 【問題3】Kaggle Notebooksからの調査
KaggleのNotebooksから様々なアイデアを見つけ出して、列挙してください。

1.オブジェクト変数の数値化、特徴量を増やす、欠損値をなくす

2.異常値(勤続1000年等）に何かしらの値を代入

3.モデルにLightGBMを使う

4.多項式で新しい特徴量を作る。（PolynomialFeatures)

【参考】

https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction

https://www.kaggle.com/jsaguiar/lightgbm-with-simple-features


### 【問題4】高い汎化性能のモデル作成
問題3で見つけたアイデアと、独自のアイデアを組み合わせ高い汎化性能のモデル作りを進めてください。

その過程として、何を行うことで、クロスバリデーションの結果がどの程度変化したかを表にまとめてください。

### 【問題5】最終的なモデルの選定
最終的にこれは良いというモデルを選び、推定した結果をKaggleに提出してスコアを確認してください。

どういったアイデアを取り入れ、どの程度のスコアになったかを記載してください。

|  | 課題１ |課題４|
| :---: | :---: | :---: | 
| モデル | ランダムフォレスト | ランダムフォレスト |
| 特徴量の数 | 4 | 15 |
| AUC算出値 | 0.549 | 0.717 |
| Kaggleスコア |0.544 |  0.697 |
|その他||オブジェクト変数を数値化<br>よりよい特徴量を使用できた|
|||異常値の処理を実施<br>適切な機械学習ができたと考える|

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_curve, auc, accuracy_score)
from sklearn.model_selection import KFold

#csvファイルの読込
df_train = pd.read_csv("application_train.csv")

#データ前処理
#オブジェクト変数をワンホットエンコーディングする。
# 例：CODE_GENDER →　CODE_GENDER_M, CODE_GENDER_F, CODE_GENDER_XNAの３列に分割して0,1化
df_train = pd.get_dummies(df_train)
#df_train.to_csv('dummies.csv',index = False)

#欠損値を0で補完する。
df_train.fillna(0, inplace=True)
#df_train.to_csv('fillna.csv',index = False)

#異常値を削除する。　勤続年数1000年をその他の数値の平均に置換
max_value = df_train["DAYS_EMPLOYED"].max()
median_value = df_train["DAYS_EMPLOYED"].median()
df_train = df_train.replace({"DAYS_EMPLOYED": {max_value: median_value}})
#print(mean_value)

#使用する特徴量 ポアソン相関係数が比較的高かったもの＋任意の特徴量の合計10個
features = ["AMT_INCOME_TOTAL", 
#"CNT_CHILDREN", 
"AMT_CREDIT", 
"DAYS_BIRTH", 
"DAYS_EMPLOYED", 
#"DAYS_REGISTRATION", 
"DAYS_ID_PUBLISH", 
"FLAG_EMP_PHONE", 
"REGION_RATING_CLIENT_W_CITY", 
"EXT_SOURCE_1", 
"EXT_SOURCE_2", 
"EXT_SOURCE_3", 
"NAME_CONTRACT_TYPE_Cash loans", 
#"NAME_CONTRACT_TYPE_Revolving loans", 
"FLAG_OWN_CAR_N",  
#"NAME_TYPE_SUITE_Children", 
"NAME_TYPE_SUITE_Family", 
"NAME_FAMILY_STATUS_Married", 
"CODE_GENDER_F", 
           ]

#trainデータの特徴量と目的変数の抽出
X = df_train.loc[:,features].values
y = df_train.loc[:,"TARGET"].values

#train, testデータへの分割
X_init_train, X_init_test, y_init_train, y_init_test = train_test_split(X, y, test_size=0.2)

#標準化
scaler = StandardScaler()
scaler.fit(X_init_train)
X_train_transformed = scaler.transform(X_init_train)
X_test_transformed = scaler.transform(X_init_test)

#クロスバリデーション
accuracy_list = []
auc_list = []
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X_train_transformed):
    X_train_cross, X_test_cross = X_train_transformed[train_index], X_train_transformed[test_index]
    y_train_cross, y_test_cross = y_init_train[train_index], y_init_train[test_index]

    #ランダムフォレストにて学習
    rf = RandomForestClassifier()
    rf.fit(X_train_cross, y_train_cross)
    
    #精度予測
    y_pred = rf.predict(X_test_transformed)
    score = accuracy_score(y_init_test, y_pred)
    accuracy_list.append(score)

    #ROC予測
    y_pred = rf.predict_proba(X_test_transformed)
    fpr, tpr, thresholds = roc_curve(y_init_test, y_pred[:,1], pos_label=1)
    auc_list.append(auc(fpr, tpr))
    
#予測精度, AUCの表示
for i, acc in enumerate(accuracy_list):
    print("No.{}, ACC : {}".format(i, acc))
print("ACCの平均: {}".format(np.mean(accuracy_list)))
for i, auc in enumerate(auc_list):
    print("No.{}, AUC : {}".format(i, auc))
print("AUCの平均: {}".format(np.mean(auc_list)))


#test.csvからテスト用データの作成（トレインデータと同様）
df_test = pd.read_csv("application_test.csv")
df_test = pd.get_dummies(df_test)
df_test.fillna(0, inplace=True)
max_value = df_test["DAYS_EMPLOYED"].max()
median_value = df_test["DAYS_EMPLOYED"].median()
df_test = df_test.replace({"DAYS_EMPLOYED": {max_value: median_value}})

#test.csvの特徴量の抽出
X_test_features = df_test.loc[:,features].values

#test.csv特徴量の標準化
scaler = StandardScaler()
scaler.fit(X_test_features)
X_test_features_transformed = scaler.transform(X_test_features)

#test.csv特徴量からデフォルト予測
y_test_pred = rf.predict_proba(X_test_features_transformed)

#以下、Kaggle提出用データの作成
df_X_test_ID = df_test.loc[:,["SK_ID_CURR"]]
df_y_pred = pd.DataFrame(y_test_pred[:,1])
df_output = pd.concat([df_X_test_ID, df_y_pred], axis=1)
df_output.columns = ["SK_ID_CURR", "TARGET"]
df_output.to_csv('home_creditdata_4.csv',index = False)


No.0, ACC : 0.9183617059330439
No.1, ACC : 0.9186218558444303
No.2, ACC : 0.9183942246719672
No.3, ACC : 0.9186218558444303
No.4, ACC : 0.9183779653025056
ACCの平均: 0.9184755215192755
No.0, AUC : 0.7169697466000684
No.1, AUC : 0.7163242996073547
No.2, AUC : 0.7180129591898027
No.3, AUC : 0.7160663969554535
No.4, AUC : 0.7179230884105645
AUCの平均: 0.7170592981526488
